### IMPORTS

In [245]:
import ealib

In [199]:
# EDGAR API requires request header
req_header = {"User-Agent": "roberto.brera.24@outlook.com"}

### COMP TICKERS

In [ ]:
# Dictionary to pandas dataframe
tickers_df = get_tickers_df(req_header)
tickers_df.head()

In [ ]:
# Examples on searches performable on this dataframe
tickers_df.iloc[133]
tickers_df[tickers_df["ticker"] == "NVDA"]
tickers_df[0:4]

In [205]:
# Search func examples
find_ticker(tickers_df, "NVDA")["cik_str"].iloc[0]
find_title_substring(tickers_df, "suzano")


,cik_str,ticker,title
752,0000909327,SUZ,Suzano S.A.


### COMP METADATA

In [ ]:
# First find the cik for some company
query_substr =  "gamestop"
query_cik = find_title_substring(tickers_df, query_substr)["cik_str"].iloc[0]

# Then use cik to get company metadata
filing_mtd_dict = get_metadata_dict(req_header, query_cik)

# filing_mtd_dict["filings"]["recent"].keys()
recent_filings_dict = filing_mtd_dict["filings"]["recent"]

# Convert to dataframe of recent filings
recent_filings_df = pd.DataFrame.from_dict(recent_filings_dict)
recent_filings_df.head()


In [270]:
# Value counts for some given column
doc_types_freq = recent_filings_df['primaryDocDescription'].value_counts()
doc_types_freq.head()

primaryDocDescription
FORM 4              283
PRIMARY DOCUMENT    181
                    145
8-K                 134
10-Q                 32
Name: count, dtype: int64

In [271]:
""" Access the contents of the filings with the following URL format
https://www.sec.gov/Archives/edgar/data/{cik}/{accession-number}/{filename}
"""

"""
Coming soon: abstraction for this procedure 
"""

query_doc_types = ["424B5"]
doc_data = filter_by_column(recent_filings_df, "primaryDocDescription", query_doc_types).iloc[2]
accession_number_stripped = doc_data["accessionNumber"].replace("-","")
filename = doc_data["primaryDocument"]

doc_url = f"https://www.sec.gov/Archives/edgar/data/{query_cik}/{accession_number_stripped}/{filename}"

response = requests.get(doc_url, headers=req_header)

# Write text to local file:
resp_file = "resp.txt"
resp_text = response.text
with open(resp_file, 'w', encoding='utf-8') as file:
    # Write the text of the response to the file
    file.write(resp_text)


In [272]:
# Use pdfkit to convert html to local pdf file
import pdfkit

options = {
    'no-images': '',  
    'disable-external-links': '',
    'disable-internal-links': ''  
}

resp_pdf = "resp.pdf"
pdfkit.from_string(resp_text, resp_pdf, options=options)


True

### COMPANY FACTS

In [211]:
comp_facts = get_companyfacts_dict(req_header, query_cik)
us_gaap_facts_dict = comp_facts["facts"]["us-gaap"]
us_gaap_facts_dict.keys()

dict_keys(['AccountsPayableCurrent', 'AccrualForTaxesOtherThanIncomeTaxesCurrent', 'AccruedIncomeTaxesCurrent', 'AccruedLiabilitiesCurrent', 'AccruedRentCurrent', 'AccumulatedAmortizationDeferredFinanceCosts', 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'AdditionalPaidInCapitalCommonStock', 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue', 'AdjustmentToAdditionalPaidInCapitalIncomeTaxEffectFromShareBasedCompensationNet', 'AdvertisingExpense', 'AllowanceForDoubtfulAccountsReceivableCurrent', 'AmortizationOfFinancingCosts', 'AmortizationOfFinancingCostsAndDiscounts', 'AmortizationOfIntangibleAssets', 'AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount', 'AssetImpairmentCharges', 'Assets', 'AssetsCurrent', 'AssetsHeldForSaleLongLivedFairValueDisclosure', 'AssetsNoncurrent', 'AssetsOfDisposalGroupIncludingDiscontinuedOperationCurrent', 'Bus

In [212]:

test_qr_substr = "revenue"
filtered_keys = find_dict_key_substr(us_gaap_facts_dict, test_qr_substr)

"""
sub_dict = {key: us_gaap_facts_dict[key] for key in filtered_keys if key in us_gaap_facts_dict}
"""
filtered_keys

['ContractWithCustomerAssetCumulativeCatchUpAdjustmentToRevenueChangeInMeasureOfProgress',
 'ContractWithCustomerLiabilityCumulativeCatchUpAdjustmentToRevenueChangeInMeasureOfProgress',
 'CostOfRevenue',
 'DeferredRevenueCurrent',
 'RevenueFromContractWithCustomerExcludingAssessedTax',
 'Revenues',
 'SalesRevenueGoodsNet']

In [213]:
us_gaap_facts_dict["Revenues"].keys()

dict_keys(['label', 'description', 'units'])

In [ ]:
# Convert to pd data frame
pd.DataFrame(us_gaap_facts_dict["Revenues"]["units"]["USD"])

### COMPANY CONCEPT

In [ ]:
rev_concept = get_companyconcept_dict(req_header, query_cik, "/us-gaap/Revenues")

# Coincides with company facts request
pd.DataFrame(rev_concept["units"]["USD"])
